# Intro 

In [9]:
import os
os.chdir('..')

Cuando trabajes con el **Protocolo de Contexto de Modelo (MCP)**, normalmente estarás trabajando con (o construyendo) **clientes o servidores**. En algunos proyectos, puede que tengas la oportunidad de construir ambos. Para comprender realmente el protocolo y aprovechar todo su potencial en tus proyectos, necesitas familiarizarte con **todos los componentes de la arquitectura MCP**.

En este notebook aprenderás sobre el **lado del consumidor** de esta arquitectura: las **aplicaciones host** y los **clientes**.

Primero, profundizaremos en las **aplicaciones host**: qué son, qué hacen y qué tipos de aplicaciones pueden beneficiarse al incorporar MCP. Luego, examinaremos el **cliente** en sí. Esto es lo que convierte a una aplicación host en un verdadero host: **alberga uno o varios clientes**, que a su vez permiten que la aplicación host se comunique con los **servidores MCP**.

Sin embargo, hay una limitación importante: **un solo cliente solo puede comunicarse con un solo servidor**. Si deseas acceder a múltiples servidores, tendrás que **iniciar varias instancias de cliente** o tener **un solo cliente que establezca conexiones breves y temporales con distintos servidores**. Analizaremos las **ventajas y desventajas** de cada enfoque, y los **casos de uso** en los que cada uno resulta más adecuado.

Después, verás un **ejemplo sencillo de una aplicación host y el cliente que alberga**. Lo revisaremos **línea por línea** para que entiendas la estructura de un cliente y cómo proporciona funcionalidad a la aplicación host. Finalmente, aprenderás sobre las **mejores prácticas** para integrar un cliente dentro de una aplicación host, garantizando que tu aplicación se mantenga **segura, confiable y con buen rendimiento**.


# La aplicación host

Tu aplicación puede ser cualquier cosa que aloje al/los cliente(s) que gestionarán las conexiones con servidores MCP, desde un script de chatbot sencillo que recibe entrada del usuario y la envía a un LLM para obtener una respuesta, hasta un IDE con todas las funciones como Cursor o Windsurf.

Gracias a la modularidad de MCP, hay muy pocas restricciones sobre lo que debe hacer tu aplicación host para soportar MCP. Solo necesita comunicarse con un LLM y alojar uno o más clientes MCP. Si planeas usar herramientas, entonces el LLM que utilice tu aplicación host también debe soportar el llamado de herramientas (tool calling).

En el ejemplo a continuación verás una aplicación host mínima. En su estado actual solo toma la entrada del usuario en un bucle constante y la pasa a un LLM. Puedes encontrar este script y todo el código de este capítulo en `ch3` en el repositorio de Github del libro.

Ejemplo: Una aplicación host simple


Aquí tenemos los inicios de un host MCP: un script muy simple que toma la entrada del usuario, la envía a un LLM (en este caso, ChatGPT de OpenAI) y muestra la respuesta. Veámoslo línea por línea.

En las líneas 1–4 importamos algunos paquetes:
`os` para cargar variables de entorno,
`openai` para utilizar la clase del cliente de OpenAI,
y `dotenv` para cargar temporalmente pares clave-valor desde un archivo hacia las variables de entorno.

Esto permite almacenar información sensible, como las claves de API, fuera del código, en un archivo llamado convencionalmente `.env`, con el formato `CLAVE=VALOR`.



___
⚠️ **Advertencia**
No subas tus archivos `.env` al control de versiones, ya que esto puede exponer tus claves de API al público, lo que podría provocar consecuencias como **uso no autorizado** y **costos elevados** si utilizas un modelo de pago por uso.

Si lo haces por accidente, inicia sesión de inmediato en el proveedor de tu API y **revoca las claves comprometidas**.
___

En las líneas 6–9, la aplicación llama a `load_dotenv()`, que se encarga de **cargar las claves en las variables de entorno**, y luego obtiene la clave cargada accediendo al diccionario `os.environ`. Esa clave se usa para **instanciar el cliente de OpenAI**.

Después, en la línea 11, se muestra un **mensaje de bienvenida** al usuario, y el código entra en un **bucle infinito**. La línea 14 solicita un *prompt* al usuario, mientras que las líneas 15–17 verifican si el mensaje contiene la palabra de salida. Si se encuentra, la aplicación imprime un mensaje de despedida y finaliza su ejecución.

Las líneas 18–28 se encargan de la **comunicación con el modelo**. Este es un patrón que verás repetidamente al realizar llamadas a modelos LLM desde código. Desde el cliente de OpenAI se accede a la función `.chat.completions.create()`, a la cual se le pasan varios parámetros, entre ellos:

* `max_tokens`, que controla el número máximo de tokens generados en la respuesta,
* `messages`, una lista de diccionarios con los mensajes y su rol (en este caso, se establece el rol como `"user"` y el contenido como el *prompt* del usuario),
* y `model`, que especifica **qué modelo** se usará para la generación.

La interfaz de *chat completions* dentro de la API de OpenAI es muy potente, y la función `create()` tiene muchos otros parámetros que puedes explorar para **ajustar el estilo o tipo de respuestas** que devuelve el modelo. Puedes consultar más información en la documentación oficial de OpenAI.

Finalmente, en las líneas 29–30, el código imprime el contenido de texto de la respuesta generada por el modelo y devuelta por el cliente de OpenAI.

A continuación, pasaremos del cliente del modelo (OpenAI) al **cliente MCP** que nuestra aplicación alojará.


# El Cliente

Cuando incorporas soporte para **MCP** en tus aplicaciones, el **cliente** es uno de los componentes más importantes que construirás y con los que trabajarás. Los clientes permiten la **comunicación entre los servidores MCP y tu aplicación**, actuando como una **interfaz** entre el servidor, tu aplicación y el modelo de lenguaje que estés utilizando.

Dado que los clientes sirven como punto de conexión entre tu aplicación y los servidores MCP, **ellos deciden qué funcionalidades del servidor soportarán**. Al momento de escribir este texto, los servidores MCP pueden ofrecer a una aplicación host los siguientes elementos:

* **Recursos (resources):** representan datos como archivos de texto, archivos de registro (*log files*), entre otros.
* **Prompts.**
* **Herramientas (tools):** fragmentos de código ejecutable que un agente o modelo puede ejecutar.
* **Muestreo (sampling):** una forma en la que el servidor puede solicitar *chat completions* al modelo de la aplicación host.
* **Imágenes (images).**
* **Contexto (context):** que proporciona capacidades internas de MCP a las herramientas.

Puedes aprender más sobre cada uno de estos en el **Capítulo 4: El Servidor**.

Además, los clientes también pueden soportar **raíces (roots)**, que definen **límites** (por ejemplo, una ubicación específica en el sistema de archivos) dentro de los cuales los servidores conectados deben operar. Estas raíces no se aplican de forma estricta, por lo que depende del servidor respetarlas y del usuario del cliente revisar cuidadosamente los servidores antes de utilizarlos.

Actualmente, MCP soporta **dos mecanismos principales de transporte**:

* **Entrada/Salida Estándar (Standard Input/Output – stdio)**
* **HTTP Transmitible (Streamable HTTP)**

En este capítulo aprenderás cómo implementar soporte para cada mecanismo oficial. En el **Capítulo 5** se profundizará en la **capa de transporte**, cómo funcionan las implementaciones oficiales y cómo podrías desarrollar tu propio sistema de transporte.

> **Nota**
> Anthropic lanzó recientemente una **versión beta del conector MCP**, que promete permitir a los usuarios acceder directamente a servidores MCP remotos mediante la **API de Mensajes del SDK de Anthropic**.
> Aunque esto podría reducir la necesidad de construir clientes personalizados, hasta el momento **solo soporta herramientas y servidores MCP remotos**, lo cual —como verás en el capítulo sobre servidores MCP— **podría representar un riesgo de seguridad**. Además, este enfoque **vincula al usuario a los modelos de Anthropic (como Claude)**, que pueden no ajustarse a todos los casos de uso.


# Diseño Básico de un Cliente

En MCP, un **cliente** se encarga de **toda la comunicación y conexión con un servidor**. Dado que las conexiones cliente-servidor son **uno a uno**, lo más recomendable es construir una **clase cliente**.

Un cliente, como mínimo, debe poder realizar las siguientes acciones:

* Conectarse a un servidor.
* Descubrir los **recursos** del servidor.
* Poner esos recursos a disposición de un **LLM**.

Más allá de estas operaciones básicas, suele ser útil implementar también:

* **Autenticación**
* **Filtrado de recursos**
* **Independencia de modelo**

En el resto de este capítulo aprenderás cómo implementar cada una de estas características y qué aportan a tus aplicaciones. Primero, esbozaremos la **interfaz de una clase cliente**.

Si sigues el repositorio de GitHub, el código de esta sección está en `client.py`, mientras que todo el código de la aplicación host está en `agent.py`. Mantendremos esta estructura durante el resto del capítulo.

```python
class MCPClient:
    def __init__(self) -> None:
        pass

    async def connect(self) -> None:
        """
        Conectar con el servidor definido en el constructor.
        """
        pass

    async def get_available_tools(self) -> list[Any]:
        """
        Recuperar las herramientas que el servidor ha puesto a disposición.
        """
        pass

    async def use_tool(self, tool_name: str, tool_args: list | None = None):
        """
        Dado un nombre de herramienta y opcionalmente una lista de argumentos, ejecutar la herramienta.
        """
        pass

    async def disconnect(self) -> None:
        """
        Liberar recursos utilizados.
        """
        pass
```

En esta clase se definieron las **firmas de 3 métodos**, además del constructor:

* `connect()` – inicializa la conexión con el servidor. La implementación dependerá del mecanismo de transporte que elijas.
* `get_available_tools()` – utiliza la conexión del cliente con el servidor para **recuperar las herramientas disponibles**.
* `use_tool()` – llama a una herramienta por su nombre, pasando los argumentos proporcionados.

Otro nombre válido para tu cliente podría ser **MCPServer**, como se hace en los ejemplos oficiales de Python, ya que desde la perspectiva de la aplicación host, el objeto instanciado **representaría un servidor MCP**. Esto puede ser confuso, así que en este ejemplo se eligió el nombre **MCPClient**. En tus proyectos, elige el que tenga más sentido para ti y tus usuarios.

Observa que este esqueleto **empieza con soporte para herramientas (tools)**. Las herramientas son uno de los **principales primitivos de MCP** y probablemente el caso de uso más popular. Permiten dar **agencia a los agentes**, **ampliar el conocimiento de los LLMs** y mucho más. Por eso empezamos implementando el soporte de herramientas, para luego avanzar a soportar todos los diferentes recursos que un servidor MCP puede ofrecer.

> **Nota:** Los tres primitivos de MCP son **tools, prompts y resources**.


## Inicializando el Cliente y Conectando a un Servidor

Antes de escribir código, debemos pensar en **cómo queremos conectarnos a un servidor**, lo que implica decidir qué **transporte** vamos a usar.

En MCP, un **transporte** es una implementación de la **capa de transporte del protocolo**, que gestiona cómo se envían los mensajes entre el cliente y el servidor.

Actualmente, MCP tiene **dos implementaciones de transporte principales**:

* **stdio**:
  Adecuado cuando se espera que los servidores se ejecuten junto a la aplicación host. Usa **entradas y salidas estándar** para la comunicación entre cliente y servidor. Es el transporte más común, ya que es sencillo de implementar y la mayoría de las aplicaciones comerciales que alojan clientes MCP esperan que el usuario instale y ejecute los servidores por su cuenta.

> **Nota:**
> El SDK de Python de MCP también incluye transporte **websocket** y **HTTP Server-Sent Events (SSE)**, aunque SSE está siendo reemplazado por **Streamable HTTP**. En este capítulo nos enfocaremos en Streamable HTTP, pero los patrones que veremos junto con la guía de compatibilidad de Anthropic son suficientes para soportar ambos transportes remotos. Todos los transportes se explorarán con más detalle en el capítulo 5.

* **Streamable HTTP**:
  Ideal para aplicaciones donde los servidores MCP **no se ejecutarán necesariamente junto a la aplicación host**, como en desarrollo de plataformas o herramientas hospedadas. Por ejemplo, si los servidores MCP están desplegados en otra máquina o en la red pública, un cliente stdio **nunca podrá comunicarse** con ellos. En ese caso, necesitas implementar soporte para **Streamable HTTP**.

> **Advertencia:**
> Cualquier transporte remoto, incluyendo Streamable HTTP, **debe ser asegurado correctamente** para evitar problemas de seguridad. Esto incluye **autenticar la conexión** y **validar los encabezados de origen**.

---

Primero, veamos el caso más simple: **conexión vía stdio**.

Al conectarte a un servidor MCP mediante stdio:

* El cliente **lanza el servidor como un subproceso**.
* Luego, **lee los mensajes desde la entrada estándar** del servidor y **envía mensajes por su salida estándar**.

El SDK de Python de MCP ya incluye todas las estructuras necesarias para construir un cliente y conectarse vía stdio.

Comencemos configurando el **constructor** de la clase `MCPClient`:


In [10]:
# client.py
from contextlib import AsyncExitStack
from typing import Any

from mcp import ClientSession

class MCPClient:
    def __init__(
        self,
        name: str,
        command: str,
        server_args: list[str],
        env_vars: dict[str, str] = None
    ) -> None:
        self.name = name  # Nombre del cliente
        self.command = command  # Comando para iniciar el servidor MCP
        self.server_args = server_args  # Argumentos adicionales del servidor
        self.env_vars = env_vars  # Variables de entorno opcionales
        self._session: ClientSession = None  # Sesión cliente MCP
        self._exit_stack: AsyncExitStack = AsyncExitStack()  # Gestión de recursos async
        self._connected: bool = False  # Estado de conexión
    ...


En este constructor se configuran varias **propiedades importantes** que permitirán conectarse e interactuar con un servidor MCP:

* **`name`**: proporciona un nombre legible para humanos al cliente instanciado. Esto es útil para **logging**, especialmente si mantienes conexiones con múltiples servidores, ya que permite identificar fácilmente el origen de cualquier registro emitido. No es obligatorio para un cliente MCP.

* Los siguientes tres parámetros se usan para **configurar el servidor**:

  * **`command`**: identifica el ejecutable que se ejecutará para iniciar el servidor. Normalmente, los servidores se ejecutan con **python** o **node** (o **npx** según la instalación local). Para muchos casos de uso, puede ser útil definir un **enum** para asegurar que se use un comando soportado.
  * **`server_args`**: lista de todos los argumentos de línea de comando que se pasan al ejecutable. Esto incluye, al menos, la **ruta al archivo del servidor**. Por ejemplo, si el archivo acepta argumentos `--port 8000 --verbose`, la lista sería `["server.py", "--port", "8000"]`.
  * **`env_vars`**: diccionario de variables de entorno. Funcionan como variables de entorno normales, y podrías incluso desempaquetar las variables de entorno del sistema con `{**os.environ}`, aunque no se recomienda porque podría exponer información sensible al servidor.

También se incluyen algunas propiedades “privadas” de **gestión de conexión**:

* **`_session`**: mantendrá la sesión del cliente.
* **`_exit_stack`**: gestiona los contextos de conexión asíncronos (más adelante veremos cómo).
* **`_connected`**: evita intentar conectarse a un servidor si ya estamos conectados.

El siguiente paso es implementar los métodos **`connect()`** y **`disconnect()`**, donde se utilizarán los **objetos de gestión de conexión** del SDK de Python de MCP para establecer y mantener la conexión con el servidor.


In [11]:
# client.py
from contextlib import AsyncExitStack
from typing import Any

from mcp import ClientSession
from mcp.client.stdio import StdioServerParameters, stdio_client

class MCPClient:
    ...
    async def connect(self) -> None:
        """
        Conectar con el servidor definido en el constructor.
        """
        if self._connected:
            raise RuntimeError("El cliente ya está conectado")

        # Configurar parámetros del servidor stdio
        server_parameters = StdioServerParameters(
            command=self.command,
            args=self.server_args,
            env=self.env_vars if self.env_vars else None
        )

        # Conectar al servidor stdio, iniciando el subproceso
        stdio_connection = await self._exit_stack.enter_async_context(
            stdio_client(server_parameters)
        )
        self.read, self.write = stdio_connection

        # Iniciar sesión del cliente MCP
        self._session = await self._exit_stack.enter_async_context(
            ClientSession(read_stream=self.read, write_stream=self.write)
        )

        # Inicializar la sesión
        await self._session.initialize()
        self._connected = True

    async def disconnect(self) -> None:
        """
        Liberar recursos utilizados y cerrar la conexión.
        """
        if self._exit_stack:
            await self._exit_stack.aclose()
            self._connected = False
            self._session = None


Esto puede parecer complejo, sobre todo si no tienes experiencia con **Python asíncrono**. Vamos a desglosar lo que hace el método `connect()` para hacerlo más comprensible:

1. Lo primero que hace `connect()` es revisar la propiedad `_connected`. Esto **previene que se intente abrir una conexión ya existente**.
2. Luego, se instancia un objeto **`StdioServerParameters`**, pasando las propiedades definidas en el constructor (`command`, `server_args`, `env_vars`).
3. El resto del código crea los **procesos y conexiones necesarias** para ejecutar y usar el servidor MCP. Esto se hace mediante **`_exit_stack`**, que contiene una instancia de `AsyncExitStack`.

> **Nota:**
> `AsyncExitStack` permite **anidar manualmente context managers asíncronos** sin usar la sintaxis `async with` tradicional. Esto es ideal porque permite:
>
> * Agregar context managers dinámicamente al stack.
> * Liberar **todos los recursos en orden con una sola llamada** (`aclose()`), lo que es muy útil para controlar cuándo se cierran los recursos, en lugar de depender de salir del scope de cada context manager.
> * Ingresar a cada context manager mediante `enter_async_context()`.

---

**Flujo de conexión con stdio:**

1. Se abre una conexión al servidor stdio, iniciando el **subproceso** donde se ejecutará la sesión MCP.
2. Se pasa una instancia de `stdio_client` creada con `server_parameters`.
3. Se desempaquetan los resultados en `self.read` y `self.write`, que representan los **streams de lectura y escritura** de la sesión.
4. Se inicia la **sesión MCP** mediante otra llamada a `enter_async_context()`, esta vez con `ClientSession(read_stream=self.read, write_stream=self.write)`.
5. La sesión se inicializa con `initialize()`, que realiza:

   * La conexión real al servidor.
   * Publica las capacidades del cliente al servidor.
   * Verifica la versión del protocolo soportada por el servidor.
   * Notifica al servidor que el cliente se ha inicializado correctamente.
6. Se marca `self._connected = True`.

En resumen, para **conectarse a un servidor stdio**:

* Crear una instancia de `StdioServerParameters`.
* Iniciar el **subproceso del servidor** en un contexto asíncrono.
* Iniciar la **sesión MCP** en otro contexto anidado y guardarla en `self._session`.
* Inicializar la sesión.

El método `disconnect()` simplemente llama a `self._exit_stack.aclose()` para cerrar **todas las conexiones** en orden, y luego establece `_connected = False` y `_session = None`. Con esto, puedes **conectarte y desconectarte limpiamente** de servidores MCP usando stdio.

---

### Conexión con Streamable HTTP

El transporte **Streamable HTTP** está diseñado principalmente para **servidores MCP remotos**. Antes se usaba HTTP + SSE, pero presentaba problemas de producción, y su soporte ha sido reemplazado por Streamable HTTP.

Al conectarte a un servidor remoto mediante Streamable HTTP:

* La conexión se realiza a través de **un endpoint único definido por el servidor**.
* Las respuestas pueden ser:

  * **HTTP estándar inmediato**: se obtiene al enviar un POST y no requiere conexión siempre abierta.
  * **Respuestas SSE en streaming** (opcional): se obtienen al enviar un GET vacío y dependen de si el servidor soporta SSE.

> **Nota:**
> Aunque Streamable HTTP reemplaza HTTP + SSE, **obtener una respuesta SSE es opcional**:
>
> * El cliente debe solicitar explícitamente la respuesta en streaming.
> * El servidor puede elegir si la soporta o no.

En el siguiente paso, implementaremos un **cliente MCP que soporte Streamable HTTP** para conectarse a servidores remotos.


Para soportar **Streamable HTTP**, nos enfocaremos nuevamente en construir el **constructor**, `connect()` y `disconnect()` de la clase `MCPClient`. Empecemos con el constructor:

```python
# client.py
from contextlib import AsyncExitStack
from typing import Callable

from mcp import ClientSession

class MCPClient:
    def __init__(self, name: str, server_url: str) -> None:
        self.name = name
        self.server_url = server_url
        self._session: ClientSession = None
        self._exit_stack = AsyncExitStack()
        self._connected: bool = False
        self._get_session_id: Callable[[], str] = None
```

Este constructor es muy similar al del cliente stdio. Como **no se inicia un proceso local**, no necesitamos `command` ni `server_args`, pero sí necesitamos la ubicación del servidor, que proporciona `server_url`. El resto de las propiedades son las mismas que en el cliente stdio.

---

La conexión y desconexión del servidor también es muy similar a lo que se hace con el cliente stdio:

```python
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

class MCPClient:
    ...
    async def connect(self, headers: dict | None = None) -> None:
        if self._connected:
            raise RuntimeError("Client is already connected")

        # Conectar al servidor Streamable HTTP
        streamable_connection = await self._exit_stack.enter_async_context(
            streamablehttp_client(url=self.server_url, headers=headers)
        )
        self.read, self.write, self._get_session_id = streamable_connection

        # Iniciar sesión MCP
        self._session = await self._exit_stack.enter_async_context(
            ClientSession(read_stream=self.read, write_stream=self.write)
        )

        # Inicializar sesión
        await self._session.initialize()
        self._connected = True

    async def disconnect(self) -> None:
        """
        Liberar recursos y cerrar la conexión.
        """
        if self._exit_stack:
            await self._exit_stack.aclose()
            self._connected = False
            self._session = None
```

La principal diferencia con el cliente stdio es que aquí **se puede pasar un parámetro opcional `headers`** junto con `server_url` a `streamablehttp_client()`. Además, la función devuelve un **callback `_get_session_id`**, que permite recuperar un session ID del servidor si lo soporta, útil para **reanudar sesiones interrumpidas**. No se necesita instanciar `StdioServerParams` porque `streamablehttp_client()` recibe directamente la URL y los headers.

Otros parámetros relevantes para un desarrollador de clientes:

* **timeout**: tiempo en segundos para que las operaciones HTTP expiren (tipo `datetime.timedelta`, por defecto 30s).
* **sse_read_timeout**: tiempo en segundos para esperar un evento adicional antes de expirar (tipo `datetime.timedelta`, por defecto 5 min).
* **auth**: maneja autenticación (`httpx.auth`).

Finalmente, para integrar este cliente en una aplicación host, se puede usar un **chatbot simplificado**, instanciando un cliente MCP (stdio o Streamable HTTP) y conectándose a un servidor ficticio que provea herramientas de calculadora como `add_two_numbers`, `subtract_two_numbers`, `multiply_two_numbers` y `divide_two_numbers`.


Para adaptar tu ejemplo a **Streamable HTTP**, los cambios principales están en cómo se instancia el cliente y cómo se conecta al servidor, ya que no necesitamos lanzar un proceso local con `uv` o `python`. El resto del flujo del chatbot se mantiene igual. Aquí te muestro un ejemplo traducido y adaptado:

```python
import asyncio
from pathlib import Path

# Suponiendo que MCPClient ya está implementado para Streamable HTTP
mcp_client = MCPClient(
    name="calculator_server_connection",
    server_url="https://my-mcp-server.example.com"
)

print("Bienvenido a tu Asistente de IA. Escribe 'adiós' para salir.")

async def main():
    # Conectar al servidor remoto
    await mcp_client.connect(headers={"Authorization": "Bearer <YOUR_TOKEN>"})
    
    while True:
        prompt = input("Tú: ")
        if prompt.lower() in ("adiós", "adios"):
            print("Asistente de IA: ¡Adiós!")
            break

        message = anthropic_client.messages.create(
            max_tokens=4096,
            messages=[{"role": "user", "content": prompt}],
            model="claude-sonnet-4-0",
        )

        for response in message.content:
            print(f"Asistente: {response.text}")

    # Desconectar del servidor
    await mcp_client.disconnect()

if __name__ == "__main__":
    asyncio.run(main())
```
